# Project 6

This project involves NLP, decision trees, bagging, boosting, and more!

---

## Load packages

You are likely going to need to install the `imdbpie` package:

    > pip install imdbpie

---

In [4]:
import os
import subprocess
import collections
import re
import csv
import json

import pandas as pd
import numpy as np
import scipy

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import psycopg2
import requests
from imdbpie import Imdb
import nltk

import urllib
from bs4 import BeautifulSoup
import nltk

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

---

## Part 1: Acquire the Data

You will connect to the IMDB API to query for movies. 

See here for documentation on how to use the package:

https://github.com/richardasaurus/imdb-pie

#### 1. Connect to the IMDB API

In [5]:
imdb = Imdb()
imdb = Imdb(anonymize=True)

#### 2. Query the top 250 rated movies in the database

In [6]:
top = imdb.top_250()

#### 3. Make a dataframe from the movie data

Keep the fields:

    num_votes
    rating
    tconst
    title
    year
    
And discard the rest

In [7]:
def movie_parser(m):
    return [m['num_votes'], m['rating'], m['tconst'], m['title'], m['year']]

movies = pd.DataFrame(np.array([movie_parser(m) for m in top]),
                      columns=['num_votes','rating','tconst','title','year'])

In [8]:
movies.dtypes

num_votes    object
rating       object
tconst       object
title        object
year         object
dtype: object

#### 3. Select only the top 100 movies

In [9]:
movies = movies[0:100]

#### 4. Get the genres and runtime for each movie and add them to the dataframe

There can be multiple genres per movie, so this will need some finessing.

In [10]:
runtimes = []
genres = []

for t in movies.tconst.values:
    title = imdb.get_title_by_id(t)
    print title.title
    runtimes.append(title.runtime)
    genres.append(title.genres)

The Shawshank Redemption
The Godfather
The Godfather: Part II
The Dark Knight
Schindler's List
12 Angry Men
Pulp Fiction
The Lord of the Rings: The Return of the King
The Good, the Bad and the Ugly
Fight Club
The Lord of the Rings: The Fellowship of the Ring
Star Wars: Episode V - The Empire Strikes Back
Forrest Gump
Inception
The Lord of the Rings: The Two Towers
One Flew Over the Cuckoo's Nest
Goodfellas
The Matrix
Seven Samurai
Star Wars
City of God
Se7en
The Silence of the Lambs
It's a Wonderful Life
The Usual Suspects
Life Is Beautiful
Léon: The Professional
Once Upon a Time in the West
Spirited Away
Saving Private Ryan
American History X
Interstellar
Casablanca
Psycho
City Lights
Indiana Jones and the Raiders of the Lost Ark
The Intouchables
Rear Window
Modern Times
The Green Mile
Terminator 2: Judgment Day
The Pianist
The Departed
Back to the Future
Whiplash
Gladiator
Memento
Apocalypse Now
The Prestige
The Lion King
Dr. Strangelove or: How I Learned to Stop Worrying and Love th

In [11]:
flat_genres = [
    item 
    for sublist in genres 
    for item in sublist
]

unique_genres = np.unique(flat_genres)

In [12]:
rows = []
for i, t in enumerate(movies.tconst):
    row = [t]
    row.extend([1 if x in genres[i] else 0 for x in unique_genres])
    rows.append(row)

In [13]:
genres = pd.DataFrame(rows, columns=['tconst']+unique_genres.tolist())
genres.head()

,tconst,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,...,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,tt0111161,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,tt0068646,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,tt0071562,0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0468569,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,tt0108052,0,0,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


In [14]:
movies = movies.merge(genres, on='tconst')

In [15]:
movies['runtime'] = runtimes
print movies.columns

Index([u'num_votes',    u'rating',    u'tconst',     u'title',      u'year',
          u'Action', u'Adventure', u'Animation', u'Biography',    u'Comedy',
           u'Crime',     u'Drama',    u'Family',   u'Fantasy', u'Film-Noir',
         u'History',    u'Horror',     u'Music',   u'Musical',   u'Mystery',
         u'Romance',    u'Sci-Fi',  u'Thriller',       u'War',   u'Western',
         u'runtime'],
      dtype='object')


In [16]:
movies['year'] = movies.year.map(lambda x: float(x))
movies['rating'] = movies.rating.map(lambda x: float(x))
movies['num_votes'] = movies.num_votes.map(lambda x: float(x))

#### 4. Write the Results to a csv

In [17]:
movies.to_csv('movies.csv', index=False, encoding = 'utf-8')

In [44]:
movies = pd.read_csv('movies.csv')

---

## Part 2: Wrangle the text data

#### 1. Scrape the reviews for the top 100 movies

*Hint*: Use a loop to scrape each page at once

In [18]:
reviews = []
for i, t in enumerate(movies.tconst.values):
    print i
    reviews.append(imdb.get_title_reviews(t, max_results=100))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


#### 2. Extract the reviews and the rating per review for each movie

*Note*: "soup" from BeautifulSoup is the html returned from all 25 pages. You'll need to either address each page individually or break them down by elements.

In [19]:
expanded = []
header = ['tconst','user_rating','review']
for i, t in enumerate(movies.tconst.values):
    for r in reviews[i]:
        row = [t, r.rating, r.text]
        expanded.append(row)
    

In [20]:
user_reviews = pd.DataFrame(expanded, columns=header)
user_reviews.head()

,tconst,user_rating,review
0,tt0111161,10.0,Why do I want to write the 234th comment on Th...
1,tt0111161,10.0,"\nCan Hollywood, usually creating things for e..."
2,tt0111161,NaN,\nI have never seen such an amazing film since...
3,tt0111161,10.0,"In its Oscar year, Shawshank Redemption (writt..."
4,tt0111161,NaN,The reason I became a member of this database ...


In [21]:
user_reviews.user_rating.unique()

array([ 10.,  nan,   8.,   7.,   9.,   6.,   3.,   1.,   5.,   4.,   2.])

In [22]:
user_reviews.dropna(inplace=True)
print user_reviews.shape

(8783, 3)


#### 3. Remove the non AlphaNumeric characters from reviews

In [23]:
import string
valid_chars = string.ascii_lowercase+' '

user_reviews['review_parsed'] = user_reviews.review.map(lambda x: ''.join([x_ for x_ in x.lower() if x_ in valid_chars]))

In [24]:
user_reviews.head()

,tconst,user_rating,review,review_parsed
0,tt0111161,10.0,Why do I want to write the 234th comment on Th...,why do i want to write the th comment on the s...
1,tt0111161,10.0,"\nCan Hollywood, usually creating things for e...",can hollywood usually creating things for ente...
3,tt0111161,10.0,"In its Oscar year, Shawshank Redemption (writt...",in its oscar year shawshank redemption written...
5,tt0111161,8.0,\nI believe that this film is the best story e...,i believe that this film is the best story eve...
6,tt0111161,10.0,\nOne of my all time favorites. Shawshank Rede...,one of my all time favorites shawshank redempt...


In [25]:
user_reviews.drop(['review'], axis=1, inplace=True)

#### 4. Calculate the top 200 ngrams from the user reviews

Use the `TfidfVectorizer` in sklearn.

Recommended parameters:

    ngram_range = (1, 2)
    stop_words = 'english'
    binary = False
    max_features = 200

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
tfidf = TfidfVectorizer(ngram_range=(1,2),
                        stop_words='english',
                        binary=False,
                        max_features=200)

review_feats = tfidf.fit_transform(user_reviews.review_parsed.values)

In [28]:
review_feats = review_feats.todense()
print review_feats.shape
review_feats_cols = tfidf.get_feature_names()
print review_feats_cols[0:20]
print review_feats[0:1]

(8783, 200)
[u'acting', u'action', u'actor', u'actors', u'actually', u'amazing', u'american', u'audience', u'away', u'bad', u'beautiful', u'believe', u'best', u'better', u'big', u'bit', u'black', u'book', u'brilliant', u'camera']
[[ 0.          0.09280123  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.30525193
   0.16705604  0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.20680228  0.          0.          0.0923789   0.          0.          0.
   0.          0.          0.14807188  0.          0.20988566  0.          0.
   0.          0.          0.          0.          0.          0.          0.
   0.          0.17537346  0.          0.          0.          0.          0.
   0.          0.          0.09830664  0.          0.          0.       

In [29]:
review_feats = pd.DataFrame(review_feats, columns=review_feats_cols)

#### 5. Merge the user reviews and ratings

In [30]:
review_feats['tconst'] = user_reviews.tconst
review_feats['user_rating'] = user_reviews.user_rating

In [45]:
movies = movies.merge(review_feats, on='tconst')

In [32]:
movies.shape

(7755, 227)

In [33]:
movies.head()

,num_votes,rating,tconst,title,year_x,Action,Adventure,Animation,Biography,Comedy,...,watching,way,wife,wonderful,work,world,year_y,years,young,user_rating
0,1661358.0,9.3,tt0111161,The Shawshank Redemption,1994.0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.086050,0.000000,0.0,0.000000,0.000000,10.0
1,1661358.0,9.3,tt0111161,The Shawshank Redemption,1994.0,0,0,0,0,0,...,0.000000,0.046514,0.080187,0.000000,0.176169,0.114183,0.0,0.000000,0.068844,10.0
2,1661358.0,9.3,tt0111161,The Shawshank Redemption,1994.0,0,0,0,0,0,...,0.130955,0.103046,0.177643,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,10.0
3,1661358.0,9.3,tt0111161,The Shawshank Redemption,1994.0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.211488,0.000000,0.326944,0.0,0.477489,0.000000,8.0
4,1661358.0,9.3,tt0111161,The Shawshank Redemption,1994.0,0,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.095678,0.0,0.186312,0.000000,10.0


In [37]:
movies.columns[0:100]

Index([     u'num_votes',         u'rating',         u'tconst',
                u'title',         u'year_x',         u'Action',
            u'Adventure',      u'Animation',      u'Biography',
               u'Comedy',          u'Crime',          u'Drama',
               u'Family',        u'Fantasy',      u'Film-Noir',
              u'History',         u'Horror',          u'Music',
              u'Musical',        u'Mystery',        u'Romance',
               u'Sci-Fi',       u'Thriller',            u'War',
              u'Western',        u'runtime',         u'acting',
               u'action',          u'actor',         u'actors',
             u'actually',        u'amazing',       u'american',
             u'audience',           u'away',            u'bad',
            u'beautiful',        u'believe',           u'best',
               u'better',            u'big',            u'bit',
                u'black',           u'book',      u'brilliant',
               u'camera',           u'ca

#### 6. Save this merged dataframe as a csv

Index([u'', u'', u'', u'', u'', u'', u'', u'', u'', u'',
       ...
       u'', u'', u'', u'', u'', u'', u'', u'', u'', u''],
      dtype='object', length=227)


In [46]:
movies.to_csv('movies_user.csv', index=False, encoding='utf-8')

---

## Part 3: Combine Tables in PostgreSQL

#### 1. Import your two .csv data files into your Postgre Database as two different tables

For ease, we can call these table1 and table2

#### 2. Connect to database and query the joined set

#### 3. Join the two tables 

#### 4. Select the newly joined table and save two copies of the into dataframes

## Part 4: Parsing and Exploratory Data Analysis

#### 1. Rename any columns you think should be renamed for clarity

#### 2. Describe anything interesting or suspicious about your data (quality assurance)

#### 3. Make four visualizations of interest to you using the data

---

## Part 5: Decision Tree Classifiers and Regressors

#### 1. What is our target attribute? 

Choose a target variable for the decision tree regressor and the classifier. 

#### 2. Prepare the X and Y matrices and preprocess data as you see fit

In [3]:
movies = pd.read_csv('movies.csv')
print movies.head(3)
print movies.columns

   num_votes  rating     tconst                     title    year  Action  \
0     1994.0  1994.0  tt0111161  The Shawshank Redemption  1994.0       0   
1     1972.0  1972.0  tt0068646             The Godfather  1972.0       0   
2     1974.0  1974.0  tt0071562    The Godfather: Part II  1974.0       0   

   Adventure  Animation  Biography  Comedy   ...     Horror  Music  Musical  \
0          0          0          0       0   ...          0      0        0   
1          0          0          0       0   ...          0      0        0   
2          0          0          0       0   ...          0      0        0   

   Mystery  Romance  Sci-Fi  Thriller  War  Western  runtime  
0        0        0       0         0    0        0     8520  
1        0        0       0         0    0        0    10500  
2        0        0       0         0    0        0    12120  

[3 rows x 26 columns]
Index([u'num_votes', u'rating', u'tconst', u'title', u'year', u'Action',
       u'Adventure', u'Ani

#### 3. Build and cross-validate your decision tree classifier

#### 4. Gridsearch optimal parameters for your classifier. Does the performance improve?

#### 5. Build and cross-validate your decision tree regressor

#### 6. Gridsearch the optimal parameters for your classifier. Does performance improve?

---

## Part 6: Elastic Net


#### 1. Gridsearch optimal parameters for an ElasticNet using the regression target and predictors you used for the decision tree regressor.


#### 2. Is cross-validated performance better or worse than with the decision trees? 

#### 3. Explain why the elastic net may have performed best at that particular l1_ratio and alpha

---

## Part 7: Bagging and Boosting: Random Forests, Extra Trees, and AdaBoost

#### 1. Load the random forest regressor, extra trees regressor, and adaboost regressor from sklearn

#### 2. Gridsearch optimal parameters for the three different ensemble methods.

#### 3. Evaluate the performance of the two bagging and one boosting model. Which performs best?

#### 4. Extract the feature importances from the Random Forest regressor and make a DataFrame pairing variable names with their variable importances.

#### 5. Plot the ranked feature importances.

#### 6.1 [BONUS] Gridsearch an optimal Lasso model and use it for variable selection (make a new predictor matrix with only the variables not zeroed out by the Lasso). 

#### 6.2 [BONUS] Gridsearch your best performing bagging/boosting model from above with the features retained after the Lasso. Does the score improve?

#### 7.1. [BONUS] Select a threshold for variable importance from your Random Forest regressor and use that to perform feature selection, creating a new subset predictor matrix.

#### 7.2 [BONUS] Using BaggingRegressor with a base estimator of your choice, test a model using the feature-selected dataset you made in 7.1

## [VERY BONUS] Part 8: PCA

#### 1. Perform a PCA on your predictor matrix

#### 2. Examine the variance explained and determine what components you want to keep based on them.

#### 3. Plot the cumulative variance explained by the ordered principal components.

#### 4. Gridsearch an elastic net using the principal components you selected as your predictors. Does this perform better than the elastic net you fit earlier?

#### 5. Gridsearch a bagging ensemble estimator that you fit before, this time using the principal components as predictors. Does this perform better or worse than the original? 

#### 6. Look at the loadings of the original predictor columns on the first 3 principal components. Is there any kind of intuitive meaning here?

Hint, you will probably want to sort by absolute value of magnitude of loading, and also only look at the obviously important (larger) ones!